# Description

---

### Introduction
To be completed...

### Corruption Process
To be completed...

---

# Libraries

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Versions

In [2]:
!python --version

Python 2.7.13 :: Anaconda 4.4.0 (x86_64)


In [3]:
items = [("Numpy", np), ("Pandas", pd), ("Matplotlib", matplotlib), ("Seaborn", sns)]
for item in items:
    print item[0] + " version: " + str(item[1].__version__)

Numpy version: 1.13.0
Pandas version: 0.20.1
Matplotlib version: 2.0.2
Seaborn version: 0.7.1


# Get Data

In [4]:
# Read from disk
data = pd.read_hdf('/Users/davidziganto/Work/data/simulated_raw_data.h5', 'table')

# DF Details

In [5]:
data.head()

,degree,age,gender,major,gpa,experience,bootcamp,github,blog,articles,hired
0,1,25,1,5,1.15,14,0,3,1,7,1
1,1,21,0,0,3.61,5,0,5,0,31,0
2,0,45,1,6,2.39,24,1,14,0,35,1
3,3,54,1,5,1.98,23,0,19,1,13,0
4,0,51,1,7,1.98,44,0,14,0,24,0


In [6]:
data.describe()

,degree,age,gender,major,gpa,experience,bootcamp,github,blog,articles,hired
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.517000,39.248200,0.500800,3.521200,2.507124,17.877400,0.491400,9.754800,0.496200,20.043200,0.425800
std,1.119983,12.394052,0.500049,2.274068,0.872693,12.935109,0.499976,6.025076,0.500036,11.784342,0.494513
min,0.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.000000,0.000000,2.000000,1.740000,7.000000,0.000000,5.000000,0.000000,10.000000,0.000000
50%,2.000000,39.000000,1.000000,4.000000,2.500000,16.000000,0.000000,10.000000,0.000000,20.000000,0.000000
75%,3.000000,50.000000,1.000000,5.000000,3.280000,27.000000,1.000000,15.000000,1.000000,30.000000,1.000000
max,3.000000,60.000000,1.000000,7.000000,4.000000,50.000000,1.000000,20.000000,1.000000,40.000000,1.000000


# Corruption Process

In [7]:
# Considered changing numbers to text but would make scoring difficult bc how numbers are coded matters, 
# so decided to avoid altogether. Will include dictionary instead.

# Change degree to strings
#data.degree.replace(range(4), ['NB','B','M','P'], inplace=True)
# Change gender to F/M
#data.gender.replace(range(2), ['F','M'], inplace=True)
# Change major to strings
#data.major.replace(range(8), ['AN','BI','BS','CH','EN','JO','MA','PS'], inplace=True)
# Change bootcamp to No/Yes
#data.bootcamp.replace(range(2), ['No','Yes'], inplace=True)
# Change blog to No/Yes
#data.blog.replace(range(2), ['No','Yes'], inplace=True)
# Change hirec to No/Yes
#data.hired.replace(range(2), ['No','Yes'], inplace=True)

### [1] Add Outliers

In [8]:
# Note: these changes do NOT impact the models but are there for future improvements

# add ridiculous age
np.random.seed(24)
age_outlier_idx = np.random.randint(low=0, high=len(data.age[data['age']==39]), size=1)
for i, _ in data.iterrows():
    if data.loc[i, 'age'] == 39 & i==age_outlier_idx:
        data.loc[i, 'age'] = int(np.random.choice(a=range(120,161), size=1))
    else:
        pass
        
# years of experience that can't possibly be true given person's age (e.g. age=30, yrs exp=28)
# is already baked in; check age & experience

### [2] Simulate Data Entry Error

In [9]:
# Note: these changes definitely affect models bc values are not possible!

np.random.seed(42)
input_error_idx = np.random.randint(low=0, high=len(data), size=3)

# GPA range is [1.0, 4.0] so add one that's too high (e.g. 4.21)
data.loc[input_error_idx[0], 'gpa'] = 4.21

# blogger var is [0, 1] but add value of 2
data.loc[input_error_idx[1], 'blog'] = 2
data.loc[input_error_idx[2], 'blog'] = 2

In [10]:
data.describe()

,degree,age,gender,major,gpa,experience,bootcamp,github,blog,articles,hired
count,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.517000,39.36360,0.500800,3.521200,2.507242,17.877400,0.491400,9.754800,0.496800,20.043200,0.425800
std,1.119983,12.79011,0.500049,2.274068,0.872884,12.935109,0.499976,6.025076,0.500839,11.784342,0.494513
min,0.000000,18.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,29.00000,0.000000,2.000000,1.740000,7.000000,0.000000,5.000000,0.000000,10.000000,0.000000
50%,2.000000,39.00000,1.000000,4.000000,2.500000,16.000000,0.000000,10.000000,0.000000,20.000000,0.000000
75%,3.000000,50.00000,1.000000,5.000000,3.280000,27.000000,1.000000,15.000000,1.000000,30.000000,1.000000
max,3.000000,143.00000,1.000000,7.000000,4.210000,50.000000,1.000000,20.000000,2.000000,40.000000,1.000000


### Generate Missing Values & Inf

In [11]:
np.random.seed(199)

# Randomly select index to corrupt
nonsense_idx = np.random.randint(low=0, high=len(data), size=1)

# simulate missing data
data.loc[np.where(data['experience'] == 18)[0][0], 'experience'] = np.nan
data.loc[np.where(data['github'] == 10)[0][0], 'github'] = np.nan

# add 'inf' data type
data.loc[nonsense_idx, 'articles'] = np.inf

In [12]:
nonsense_idx

array([3492])

In [13]:
data.isnull().sum()

degree        0
age           0
gender        0
major         0
gpa           0
experience    1
bootcamp      0
github        1
blog          0
articles      0
hired         0
dtype: int64

In [14]:
np.isinf(data.articles).sum()

1

In [15]:
np.where(data.isnull())

(array([11, 12]), array([5, 7]))

In [16]:
data.iloc[12]

degree         1.00
age           28.00
gender         0.00
major          3.00
gpa            1.55
experience    10.00
bootcamp       0.00
github          NaN
blog           1.00
articles      19.00
hired          1.00
Name: 12, dtype: float64

In [17]:
np.where(data==np.inf)

(array([3492]), array([9]))

### Write To Disk

In [18]:
# Write to disk
data.to_hdf('/Users/davidziganto/Work/data/simulated_messy_data.h5',
                'table',
                mode='w', 
                append=True, 
                complevel=9, 
                complib='blosc', 
                fletcher32=True)